In [4]:
from pathlib import Path
import pandas as pd
import json
from tqdm import tqdm

In [44]:
path = Path('.')
path.resolve()

PosixPath('/mnt/d/CloudStorage/York Research/6124 Data Mining/Project coding/book-recommendations')

In [45]:
all_books = []
with open(path/'data/meta_small.json', 'r') as f:
    lines = f.read().splitlines()
    for line in tqdm(lines):
        all_books.append(json.loads(line))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:08<00:00, 11362.35it/s]


In [46]:
df = pd.DataFrame(all_books)

In [110]:
df.head()

,book_id,title,author_id,author_name,description,genre_1,genre_2,genre_3
0,5333265,W.C. Fields: A Life on Film,604031,Ronald J. Fields,,"history, historical fiction, biography",,
1,1333909,Good Harbor,626222,Anita Diamant,"Anita Diamant's international bestseller ""The ...",fiction,"history, historical fiction, biography",
2,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",10333,Barbara Hambly,Omnibus book club edition containing the Ladie...,"fantasy, paranormal",fiction,poetry
3,6066819,Best Friends Forever,9212,Jennifer Weiner,Addie Downs and Valerie Adler were eight when ...,fiction,romance,"mystery, thriller, crime"
4,287140,Runic Astrology: Starcraft and Timekeeping in ...,149918,Nigel Pennick,,non-fiction,,


In [48]:
df = df[['book_id', 'title','authors','description']]

In [49]:
df.head()

,book_id,title,authors,description
0,5333265,W.C. Fields: A Life on Film,"[{'author_id': '604031', 'role': ''}]",
1,1333909,Good Harbor,"[{'author_id': '626222', 'role': ''}]","Anita Diamant's international bestseller ""The ..."
2,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","[{'author_id': '10333', 'role': ''}]",Omnibus book club edition containing the Ladie...
3,6066819,Best Friends Forever,"[{'author_id': '9212', 'role': ''}]",Addie Downs and Valerie Adler were eight when ...
4,287140,Runic Astrology: Starcraft and Timekeeping in ...,"[{'author_id': '149918', 'role': ''}]",


In [50]:
#Parse authors
def parse_author(author_string):
    for i in author_string:
        if i['role'] == '':
            return i['author_id']
    return ''

In [51]:
df['author_id'] = df.apply(lambda row: parse_author(row['authors']), axis = 1)

In [56]:
book_by_genre = []
with open(path/'data/goodreads_book_genres_initial.json', 'r') as f:
    lines = f.read().splitlines()
    for line in tqdm(lines):
        book_by_genre.append(json.loads(line))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2360655/2360655 [02:07<00:00, 18482.67it/s]


In [57]:
genre_df = pd.DataFrame(book_by_genre)

In [58]:
genre_df

,book_id,genres
0,5333265,"{'history, historical fiction, biography': 1}"
1,1333909,"{'fiction': 219, 'history, historical fiction,..."
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri..."
4,287140,{'non-fiction': 3}
...,...,...
2360650,3084038,"{'non-fiction': 5, 'history, historical fictio..."
2360651,26168430,"{'mystery, thriller, crime': 4, 'children': 1,..."
2360652,2342551,"{'poetry': 14, 'children': 7, 'young-adult': 1..."
2360653,22017381,"{'romance': 13, 'mystery, thriller, crime': 2}"


In [61]:
df.head()

,book_id,title,authors,description,author_id
0,5333265,W.C. Fields: A Life on Film,"[{'author_id': '604031', 'role': ''}]",,604031
1,1333909,Good Harbor,"[{'author_id': '626222', 'role': ''}]","Anita Diamant's international bestseller ""The ...",626222
2,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","[{'author_id': '10333', 'role': ''}]",Omnibus book club edition containing the Ladie...,10333
3,6066819,Best Friends Forever,"[{'author_id': '9212', 'role': ''}]",Addie Downs and Valerie Adler were eight when ...,9212
4,287140,Runic Astrology: Starcraft and Timekeeping in ...,"[{'author_id': '149918', 'role': ''}]",,149918


In [64]:
df = df.merge(genre_df, on='book_id', how='left')

In [77]:
print(df.iloc[3]['genres'])
for genre in sorted(df.iloc[3]['genres'], key = lambda x:x[1]):
    print(genre)

{'fiction': 555, 'romance': 23, 'mystery, thriller, crime': 10}
fiction
romance
mystery, thriller, crime


In [80]:
def parse_top_genres(genre_dict):
    genres = ['','','']
    for i, genre in enumerate(sorted(genre_dict, key = lambda x:x[1])[:3]):
        genres[i] = genre
    return genres

In [81]:
df['top_genres'] = df.apply(lambda row: parse_top_genres(row['genres']), axis = 1)

In [83]:
df['genre_1'] = df.apply(lambda row: row['top_genres'][0], axis = 1)
df['genre_2'] = df.apply(lambda row: row['top_genres'][1], axis = 1)
df['genre_3'] = df.apply(lambda row: row['top_genres'][2], axis = 1)

In [88]:
author_name = []
with open(path/'data/goodreads_book_authors.json', 'r') as f:
    lines = f.read().splitlines()
    for line in tqdm(lines):
        author_name.append(json.loads(line))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 829529/829529 [00:06<00:00, 136108.70it/s]


In [89]:
author_df = pd.DataFrame(author_name)

In [94]:
author_df = author_df[['author_id', 'name']]

In [95]:
df = df.merge(author_df, on='author_id', how='left')

In [97]:
df.rename(columns = {'name':'author_name'}, inplace = True)

In [99]:
df.columns

Index(['book_id', 'title', 'authors', 'description', 'author_id', 'genres',
       'top_genres', 'genre_1', 'genre_2', 'genre_3', 'author_name'],
      dtype='object')

In [100]:
df = df[['book_id', 'title','author_id', 'author_name','description', 'genre_1', 'genre_2', 'genre_3']]

### Ready to process and convert to embedding

In [105]:
df

,book_id,title,author_id,author_name,description,genre_1,genre_2,genre_3
0,5333265,W.C. Fields: A Life on Film,604031,Ronald J. Fields,,"history, historical fiction, biography",,
1,1333909,Good Harbor,626222,Anita Diamant,"Anita Diamant's international bestseller ""The ...",fiction,"history, historical fiction, biography",
2,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",10333,Barbara Hambly,Omnibus book club edition containing the Ladie...,"fantasy, paranormal",fiction,poetry
3,6066819,Best Friends Forever,9212,Jennifer Weiner,Addie Downs and Valerie Adler were eight when ...,fiction,romance,"mystery, thriller, crime"
4,287140,Runic Astrology: Starcraft and Timekeeping in ...,149918,Nigel Pennick,,non-fiction,,
...,...,...,...,...,...,...,...,...
99995,36110147,Det växte ett träd i Brooklyn - Del 2,5053136,Betty Smith,,,,
99996,24679885,Shock: A Virals Short Story,26372,Kathy Reichs,A riveting short story that takes us back to t...,"fantasy, paranormal",children,fiction
99997,27883170,The Maiden Thief,175855,Melissa Marr,"""The Maiden Thief"" by Melissa Marr is a dark f...","fantasy, paranormal",fiction,"history, historical fiction, biography"
99998,35992614,Caged Warrior (Underground Fighters #1),15428911,Aislinn Kearns,Meet the Underground Fighters--men battling it...,romance,,


In [106]:
df.author_name.value_counts()

Agatha Christie       187
Stephen King          167
Anonymous             103
James Patterson        92
Nora Roberts           90
                     ... 
Winifred L. Frazer      1
Adrian Thomas           1
C.A. Green              1
Ben Mariner             1
Lenore Hart             1
Name: author_name, Length: 61526, dtype: int64

In [108]:
df.genre_1.value_counts()

fiction                                   25860
fantasy, paranormal                       22884
                                          16996
history, historical fiction, biography    12985
children                                   6897
non-fiction                                6857
romance                                    3990
poetry                                     1131
comics, graphic                            1030
mystery, thriller, crime                    979
young-adult                                 391
Name: genre_1, dtype: int64